In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt``
import seaborn as sns
from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm
import arviz as az
import pymc as pm
from sklearn.metrics import roc_curve, roc_auc_score


In [ ]:
#project_dir = r'./sample_data'
#os.chdir(project_dir)

merged4 = pd.read_csv('merged4.csv')

<ipython-input-4-581d5b7bc851>:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  merged4 = pd.read_csv('merged4.csv')


In [ ]:
!pip install pymc-bart

In [ ]:
merged4

,defender_carrier_dist,defender_distance_to_football,rho,force,TME,tackle_event,total_energy,is_defensive_player
0,11.013632,9.256266,136.06768,418.07892,-0.166554,0.0,129.529087,1.0
1,10.246878,8.688046,195.32296,486.11276,0.290935,0.0,229.789344,1.0
2,9.464486,8.500617,242.50772,460.87440,-0.30649,0.0,322.187614,1.0
3,8.646531,8.557009,296.27640,424.66284,-0.697373,0.0,449.600623,1.0
4,7.812074,8.590053,334.68260,350.04508,0.459627,0.0,551.141015,1.0
...,...,...,...,...,...,...,...,...
84453,9.690593,10.340377,621.20595,47.15724,-0.6030926752812964,0.0,2132.190874,1.0
84454,8.739611,9.675770,626.64717,43.52976,-0.6083752388604027,0.0,2169.564815,1.0
84455,7.824398,8.856415,625.74030,54.41220,-0.986305031352384,0.0,2164.947853,1.0
84456,6.907807,7.910512,621.20595,76.17708,-0.8434515455780064,0.0,2137.260234,1.0


In [ ]:
# Assuming 'merged4' is your DataFrame

# Remove rows with any missing values
merged4_clean = merged4.dropna()

# Check the result
print("After removing missing values:")
print(merged4_clean.isnull().sum())


After removing missing values:
defender_carrier_dist            0
defender_distance_to_football    0
rho                              0
force                            0
TME                              0
tackle_event                     0
total_energy                     0
is_defensive_player              0
dtype: int64


In [ ]:
merged4_clean.shape
merged4_clean

,defender_carrier_dist,defender_distance_to_football,rho,force,TME,tackle_event,total_energy,is_defensive_player
0,11.013632,9.256266,136.06768,418.07892,-0.166554,0.0,129.529087,1.0
1,10.246878,8.688046,195.32296,486.11276,0.290935,0.0,229.789344,1.0
2,9.464486,8.500617,242.50772,460.87440,-0.30649,0.0,322.187614,1.0
3,8.646531,8.557009,296.27640,424.66284,-0.697373,0.0,449.600623,1.0
4,7.812074,8.590053,334.68260,350.04508,0.459627,0.0,551.141015,1.0
...,...,...,...,...,...,...,...,...
84452,10.634971,11.031092,616.67160,53.50533,0.8881607400271516,0.0,2101.533851,1.0
84453,9.690593,10.340377,621.20595,47.15724,-0.6030926752812964,0.0,2132.190874,1.0
84454,8.739611,9.675770,626.64717,43.52976,-0.6083752388604027,0.0,2169.564815,1.0
84455,7.824398,8.856415,625.74030,54.41220,-0.986305031352384,0.0,2164.947853,1.0


In [ ]:
# BART model
RANDOM_SEED = 1000

import pymc as pm
import pymc_bart as pmb  # Make sure pymc_bart is installed

# Assuming merged4 and its columns are already defined
X = merged4_clean[['defender_carrier_dist', 'defender_distance_to_football', 'rho', 'force', 'TME', 'total_energy']].values
y = merged4_clean['tackle_event'].values

with pm.Model() as bart_tackle_event:
    # BART prior
    μ = pmb.BART("μ", X, y, m=15)  # m is the number of trees, adjust as needed

    # Logistic link function
    p = pm.math.sigmoid(μ)

    # Bernoulli likelihood for binary outcome
    y_obs = pm.Bernoulli("y_obs", p=p, observed=y)

    # Sampling with specified number of draws
    trace_tackle_event_bart = pm.sample(700, random_seed=RANDOM_SEED)

# You can then analyze the trace as needed


In [ ]:
#with bart_tackle_event:  # This is your PyMC model
#    ppc = pm.sample_posterior_predictive(trace_tackle_event_bart, var_names=["y_obs"])

#az.plot_ppc(az.from_pymc(trace_tackle_event_bart, posterior_predictive=ppc))

# Trace plots
#az.plot_trace(trace_tackle_event_bart)

# Model comparison
waic = az.waic(trace_tackle_event_bart)
loo = az.loo(trace_tackle_event_bart)

TypeError: ignored

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import numpy as np
import pymc as pm

# Assuming merged4_clean and its columns are already defined
X_merged4 = merged4_clean.drop(['tackle_event', 'is_defensive_player'], axis=1)
y_merged4 = merged4_clean['tackle_event']
X_train, X_test, y_train, y_test = train_test_split(X_merged4, y_merged4, test_size=0.3, random_state=42)

# Standardize the test data
X_test_std = (X_test - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)

# Generate posterior predictive samples for the test set
with bart_tackle_event:
    ppc = pm.sample_posterior_predictive(trace_tackle_event_bart)

# Calculate predicted probabilities

predicted_probs = ppc.posterior_predictive.y_obs.mean(dim=["chain", "draw"]).values
predicted_probs = np.ravel(predicted_probs)

# Ensure the lengths match
assert len(predicted_probs) == len(y_test), "Lengths of predicted probabilities and test labels must match"

# Print predicted probabilities and their length
print(predicted_probs); print('Predicted lengths = ', len(predicted_probs))
print('\n')
# Print test labels and their length
print(y_test); print('Test lengths = ', len(y_test))

# Calculate and print ROC AUC Score
roc_auc = roc_auc_score(y_test, predicted_probs)
print("ROC AUC Score:", roc_auc)


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3430: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3571: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


AssertionError: ignored

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

# Train-test split with indices
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
    X_merged4, y_merged4, np.arange(len(y_merged4)), test_size=0.3, random_state=42)

# Assuming you already have 'predicted_probs' from your BART model

# Select predicted probabilities for the test set
predicted_probs_test = predicted_probs[test_indices]

# Check lengths to ensure compatibility
print("Length of predicted probabilities for test set:", len(predicted_probs_test))
print("Length of test labels:", len(y_test))

# Calculate ROC AUC if lengths match
if len(predicted_probs_test) == len(y_test):
    fpr, tpr, _ = roc_curve(y_test, predicted_probs_test)
    roc_auc = auc(fpr, tpr)
    print('ROC AUC Score:', roc_auc)
else:
    print("The lengths of predicted probabilities and test labels do not match.")


Length of predicted probabilities for test set: 110586
Length of test labels: 110586
ROC AUC Score: 0.5477598012434417


In [ ]:
#waic = az.waic(trace_tackle_event_bart)
loo = az.loo(trace_tackle_event_bart)

TypeError: ignored

In [ ]:
az.plot_trace(trace_tackle_event_bart, var_names=['μ'])

KeyboardInterrupt: ignored

Error in callback <function _draw_all_if_interactive at 0x7a3331001fc0> (for post_execute):


KeyboardInterrupt: ignored

Error in callback <function flush_figures at 0x7a3331001240> (for post_execute):


KeyboardInterrupt: ignored

In [ ]:
az.rhat(trace_tackle_event_bart)


/usr/local/lib/python3.10/dist-packages/arviz/utils.py:184: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  numba_fn = numba.jit(**self.kwargs)(self.function)


<xarray.Dataset>
Dimensions:  (μ_dim_0: 368617)
Coordinates:
  * μ_dim_0  (μ_dim_0) int64 0 1 2 3 4 5 ... 368612 368613 368614 368615 368616
Data variables:
    μ        (μ_dim_0) float64 1.005 1.02 1.046 1.011 ... 0.996 1.009 0.994

In [ ]:
az.plot_rank(trace_tackle_event_bart)


In [ ]:
import arviz as az

# Specify the variable names you want to create rank plots for
varnames_to_plot = ['μ']  # Replace 'μ' with the correct variable name if needed

# Create rank plots for the specified variables
az.plot_rank(trace_tackle_event_bart.posterior, var_names=varnames_to_plot)
